In [1]:
import subprocess
import sys

packages = ['nltk', 'spacy', 'unidecode', 'numpy', 'pandas']
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("Bibliothèques installées avec succès")

✓ Bibliothèques installées avec succès


In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import spacy
from pathlib import Path

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

try:
    nlp = spacy.load('fr_core_news_sm')
except OSError:
    print("Téléchargement du modèle spaCy français...")
    subprocess.check_call([sys.executable, '-m', 'spacy', 'download', 'fr_core_news_sm'])
    nlp = spacy.load('fr_core_news_sm')

print("Imports et modèles chargés")

Téléchargement du modèle spaCy français...
Imports et modèles chargés


In [19]:
def load_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        return text
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} n'existe pas")
        return ""
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {e}")
        return ""

text = load_text_file('entree.txt')
print(f"Texte chargé: {len(text)} caractères")
print("\n--- Premier 200 caractères ---")
print(text[:200] + "...")

Texte chargé: 1616 caractères

--- Premier 200 caractères ---
OH OH OH, Noel arrive, et qui dit Noel dit ... nourriture !
Jeudi 18 décembre prochain, à partir de midi, nous vous proposons un grand buffet collectif de noel pour fêter ensemble vos départs en vacan...


In [25]:
def tokenize_words1(text):
    tokens = word_tokenize(text, language='french')
    return tokens

words = tokenize_words1(text)
print(f"Nombre de mots: {len(words)}")
print(f"Premiers 20 mots: {words[:20]}")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/french/[0m

  Searched in:
    - 'C:\\Users\\thoma/nltk_data'
    - 'c:\\Users\\thoma\\anaconda3\\nltk_data'
    - 'c:\\Users\\thoma\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\thoma\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\thoma\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\thoma\\nltk_data'
**********************************************************************


In [26]:
def tokenize_words(text):
    print(f"Ressource NLTK manquante")
    import re
    tokens = re.findall(r'\b\w+\b|[.,!?;:\-]', text)
    return tokens

words = tokenize_words(text)
print(f"Nombre de mots: {len(words)}")
print(f"Premiers 20 mots: {words[:20]}")

Ressource NLTK manquante
Nombre de mots: 316
Premiers 20 mots: ['OH', 'OH', 'OH', ',', 'Noel', 'arrive', ',', 'et', 'qui', 'dit', 'Noel', 'dit', '.', '.', '.', 'nourriture', '!', 'Jeudi', '18', 'décembre']


In [16]:
def tokenize_sentences(text):
    sentences = re.split(r'\.\.\.|[.!?]\s*', text)

    return sentences

sentences = tokenize_sentences(text)
print(f"Nombre de phrases: {len(sentences)}")
print("\n--- Premières 3 phrases ---")
for i, sent in enumerate(sentences[:3], 1):
    print(f"{i}. {sent}")

Nombre de phrases: 13

--- Premières 3 phrases ---
1. OH OH OH, Noel arrive, et qui dit Noel dit 
2.  nourriture 
3. Jeudi 18 décembre prochain, à partir de midi, nous vous proposons un grand buffet collectif de noel pour fêter ensemble vos départs en vacances d'hiver après un premier semestre intense 


In [ ]:
def tokenize_bigrams(text):
    ngrams = []
    trigrammes = re.findall(r'(\w+)-(\w+)-(\w+)', text)
    for tri in trigrammes:
        ngrams.append(tri)  
    
    bigrammes = re.findall(r'(\w+)-(\w+)(?!-)', text)
    for bi in bigrammes:
        ngrams.append(bi) 
    
    return ngrams

bigrams = tokenize_bigrams(text)
print(f"Nombre de bigrammes/trigrammes (séparés par -): {len(bigrams)}")

print (f"les 10 premiers bi/tri grammes: {bigrams[:10]}")


Nombre de bigrammes/trigrammes (séparés par -): 6
les 10 premiers bi/tri grams: [('arc', 'en', 'ciel'), ('New', 'York'), ('wagon', 'restaurant'), ('chou', 'fleur'), ('arc', 'e'), ('n', 'ciel')]


In [38]:
def normalize_text(text):
    import unicodedata
    text = text.lower()
    
    text_nfd = unicodedata.normalize('NFD', text)
    text_without_accents = ''.join(char for char in text_nfd 
                                    if unicodedata.category(char) != 'Mn')
    nombres_texte = {
        '0': 'zero', '1': 'un', '2': 'deux', '3': 'trois', '4': 'quatre',
        '5': 'cinq', '6': 'six', '7': 'sept', '8': 'huit', '9': 'neuf'
    }
    
    text_normalized = ''
    for char in text_without_accents:
        if char.isdigit():
            text_normalized += nombres_texte[char] + ' '
        else:
            text_normalized += char
    
    return text_normalized.strip()

print("TEST \n")

sample_text = "Noël est le 25 décembre! C'est une fêtE avec Événements."
normalized = normalize_text(sample_text)

print(f"Original:  {sample_text}")
print(f"Normalisé: {normalized}")

text_normalized = normalize_text(text)
print(f"Nombre de caractères: {len(text)} → {len(text_normalized)}")
print(f"\nPremier 200 caractères normalisés:")
print(text_normalized[:200] + "...")


TEST 

Original:  Noël est le 25 décembre! C'est une fêtE avec Événements.
Normalisé: noel est le deux cinq  decembre! c'est une fete avec evenements.
Nombre de caractères: 1616 → 1634

Premier 200 caractères normalisés:
oh oh oh, noel arrive, et qui dit noel dit ... nourriture !
jeudi un huit  decembre prochain, a partir de midi, nous vous proposons un grand buffet collectif de noel pour feter ensemble vos departs en...


In [41]:
french_stopwords = {
    'le', 'la', 'les', 'de', 'des', 'un', 'une', 'et', 'ou', 'mais', 'donc',
    'car', 'par', 'pour', 'avec', 'sans', 'sur', 'sous', 'dans', 'entre',
    'avant', 'apres', 'pendant', 'lors', 'qui', 'que', 'quoi', 'quel', 'quelle',
    'dont', 'duquel', 'auquel', 'a', 'au', 'aux', 'ce', 'cet', 'cette', 'ces',
    'je', 'tu', 'il', 'elle', 'nous', 'vous', 'ils', 'elles', 'on', 'mon', 'ton',
    'son', 'notre', 'votre', 'leur', 'mes', 'tes', 'ses', 'nos', 'vos', 'leurs',
    'moi', 'toi', 'lui', 'me', 'te', 'se', 'nous', 'vous', 'leur', 'est', 'etre',
    'avoir', 'avoir', 'faire', 'aller', 'venir', 'pouvoir', 'devoir', 'vouloir',
    'c', 'est', 'd', 'l', 'j', 's', 't', 'n', 'm', 'y', 'en', 'a', 'as', 'oh'
}

def clean_tokens(tokens, remove_stopwords=True, remove_punctuation=True):
    import string
    
    cleaned = []
    
    for token in tokens:
        if remove_punctuation:
            if token in string.punctuation or all(c in string.punctuation for c in token):
                continue
            token = ''.join(char for char in token if char not in string.punctuation)
        
        if remove_stopwords:
            if token.lower() in french_stopwords:
                continue
        
        if token.strip():
            cleaned.append(token)
    
    return cleaned

print("netteoyage\n")

tokens_to_clean = tokenize_words(text_normalized)

print(f"Tokens avant nettoyage: {len(tokens_to_clean)}")
print(f"Premiers 20 tokens: {tokens_to_clean[:20]}\n")

cleaned_tokens = clean_tokens(tokens_to_clean, remove_stopwords=True, remove_punctuation=True)

print(f"Tokens après nettoyage: {len(cleaned_tokens)}")
print(f"Premiers 20 tokens nettoyés: {cleaned_tokens[:20]}\n")

print("STATS")
print(f"Tokens supprimés: {len(tokens_to_clean) - len(cleaned_tokens)}")
print(f"Ratio de compression: {(len(tokens_to_clean) - len(cleaned_tokens)) / len(tokens_to_clean) * 100:.1f}%")


netteoyage

Ressource NLTK manquante
Tokens avant nettoyage: 319
Premiers 20 tokens: ['oh', 'oh', 'oh', ',', 'noel', 'arrive', ',', 'et', 'qui', 'dit', 'noel', 'dit', '.', '.', '.', 'nourriture', '!', 'jeudi', 'un', 'huit']

Tokens après nettoyage: 156
Premiers 20 tokens nettoyés: ['noel', 'arrive', 'dit', 'noel', 'dit', 'nourriture', 'jeudi', 'huit', 'decembre', 'prochain', 'partir', 'midi', 'proposons', 'grand', 'buffet', 'collectif', 'noel', 'feter', 'ensemble', 'departs']

STATS
Tokens supprimés: 163
Ratio de compression: 51.1%


In [44]:

def simple_stemmer(word):
    suffixes = [
        'ement', 'ment', 'tion', 'sion',
        'able', 'ible', 'ateur', 'atrice',
        'eux', 'euse', 'ais', 'ait', 'aient',
        'és', 'é', 'e', 's',
        'et', 'elle', 'ette',
        'ment',
        'ant', 'ant', 'ent'
    ]
    
    word_lower = word.lower()
    
    for suffix in sorted(suffixes, key=len, reverse=True):
        if word_lower.endswith(suffix) and len(word_lower) > len(suffix) + 2:
            return word_lower[:-len(suffix)]
    
    return word_lower

def stem_words_list(words):
    return [simple_stemmer(word) for word in words]


def lemmatize_text_full(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return lemmas



print("STEMMING vs LEMMATISATION")

test_words = [
    'courant', 'courants', 'couru', 'courserait', 'coureur', 'coureurs',
    'marcher', 'marcheurs', 'marche', 'marchant', 'marches',
    'livre', 'livres', 'livreur', 'livreurs', 'livraison',
    'intelligent', 'intelligente', 'intelligents', 'intelligences',
    'rapidement', 'rapide', 'rapidité', 'rapidités'
]

print("\nTEST SUR DES MOTS SPÉCIFIQUES\n")
print(f"{'Mot Original':<20} | {'Stemming':<20} | {'Lemmatisation (spaCy)':<25}")

for word in test_words:
    stem = simple_stemmer(word)
    lemma_result = lemmatize_text_full(word)
    lemma = lemma_result[0] if lemma_result else word
    print(f"{word:<20} | {stem:<20} | {lemma:<25}")




STEMMING vs LEMMATISATION

TEST SUR DES MOTS SPÉCIFIQUES

Mot Original         | Stemming             | Lemmatisation (spaCy)    
courant              | cour                 | courir                   
courants             | courant              | courant                  
couru                | couru                | courir                   
courserait           | courser              | courser                  
coureur              | coureur              | coureur                  
coureurs             | coureur              | coureur                  
marcher              | marcher              | marcher                  
marcheurs            | marcheur             | marcheur                 
marche               | march                | marche                   
marchant             | march                | marchant                 
marches              | marche               | marches                  
livre                | livr                 | livre                    
livres

In [46]:
stemmed_tokens = stem_words_list(cleaned_tokens)

print(f"Tokens nettoyés: {len(cleaned_tokens)}")
print(f"Tokens après stemming: {len(stemmed_tokens)}")
print(f"Premiers 15 tokens nettoyés: {cleaned_tokens[:15]}")
print(f"Premiers 15 après stemming: {stemmed_tokens[:15]}\n")

lemmatized_tokens = lemmatize_text_full(text_normalized)
lemmatized_cleaned = [token.lemma_ for token in nlp(text_normalized) 
                      if token.text.lower() in [t.lower() for t in cleaned_tokens]]

print(f"Tokens après lemmatisation: {len(lemmatized_cleaned)}")
print(f"Premiers 15 après lemmatisation: {lemmatized_cleaned[:15]}\n")

unique_original = len(set(cleaned_tokens))
unique_stemmed = len(set(stemmed_tokens))
unique_lemmatized = len(set(lemmatized_cleaned))

print(f"\nMots uniques dans les tokens nettoyés: {unique_original}")
print(f"Mots uniques après stemming: {unique_stemmed} (réduction de {((unique_original - unique_stemmed) / unique_original * 100):.1f}%)")
print(f"Mots uniques après lemmatisation: {unique_lemmatized} (réduction de {((unique_original - unique_lemmatized) / unique_original * 100):.1f}%)")


Tokens nettoyés: 156
Tokens après stemming: 156
Premiers 15 tokens nettoyés: ['noel', 'arrive', 'dit', 'noel', 'dit', 'nourriture', 'jeudi', 'huit', 'decembre', 'prochain', 'partir', 'midi', 'proposons', 'grand', 'buffet']
Premiers 15 après stemming: ['noel', 'arriv', 'dit', 'noel', 'dit', 'nourritur', 'jeudi', 'huit', 'decembr', 'prochain', 'partir', 'midi', 'proposon', 'grand', 'buff']

Tokens après lemmatisation: 148
Premiers 15 après lemmatisation: ['noel', 'arriv', 'dire', 'noel', 'dire', 'nourriture', 'jeudi', 'huit', 'decembre', 'prochain', 'partir', 'midi', 'proposer', 'grand', 'buffet']


Mots uniques dans les tokens nettoyés: 126
Mots uniques après stemming: 125 (réduction de 0.8%)
Mots uniques après lemmatisation: 112 (réduction de 11.1%)
